- Identificar protocolos de comunicacion mas usados.
- Ver cuales son los mas seguros.
- A que tipo de ataque son mas propensos estos protocolos.

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


UNSW_NB15 e IOT_TON (Network) datasets: https://research.unsw.edu.au/projects/unsw-nb15-dataset


In [122]:
network1= pd.read_csv("data/Network_dataset_1.csv")
# Hay datos multi valores en services en este data set, esto es para separarlo
network1['service']=network1['service'].str.split(";")
# Esto separa los valores multivalores de una fila en la columna service y los separa
network1=network1.explode('service')
network1.head()

C:\Users\cesar\AppData\Local\Temp\ipykernel_20396\3451211571.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  network1= pd.read_csv("data/Network_dataset_1.csv")


,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [123]:
network21= pd.read_csv("data/Network_dataset_21.csv")
network21.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1556351863,192.168.1.35,45133,192.168.1.1,53,udp,dns,0.000008,80,112,...,0,0,-,-,-,-,-,-,1,xss
1,1556351863,192.168.1.35,45606,176.28.50.165,80,tcp,http,1.208269,380,724,...,0,0,-,-,-,-,-,-,1,xss
2,1556351863,192.168.1.35,50797,192.168.1.1,53,udp,dns,0.008457,80,112,...,0,0,-,-,-,-,-,-,1,xss
3,1556351863,192.168.1.35,37234,52.28.231.150,80,tcp,http,1.022216,2074,1211,...,0,0,-,-,-,-,-,-,1,xss
4,1556351863,192.168.1.35,37222,52.28.231.150,80,tcp,http,1.071203,1816,1125,...,0,0,-,-,-,-,-,-,1,xss


In [124]:
unsw= pd.read_csv("data/UNSW_NB15_training-set.csv")
unsw.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


# Transformaciones:

- Seleccionamos las categorias que vamos a usar:
  - Network (proto, service, duration, src_bytes, dst_bytes, conn_state, type, label)
  - UNSW (proto, service, dur, sbytes, dbytes, state, attack_cat, label)
- Renombrar las categorias.
- Estandarizar el estado de las conexiones.
- Unificar valores de tipo de ataque.
- Porcentaje de ataques segun el protocolo.
- Porcentaje de ataques segun el servicio.

# Resultados:

- Ranking de ataque por porcentaje de protocolo.
- Ranking de ataque por porcentaje de servicio.

In [125]:
network_columns = ["proto", "service", "duration", "src_bytes", "dst_bytes", "conn_state", "type", "label"]
unsw_columns = ["proto", "service", "dur", "sbytes", "dbytes", "state", "attack_cat", "label"]

In [126]:
network1 = network1[network_columns]
network1.rename(columns={"conn_state": "connection"}, inplace=True)
network1.head()

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,tcp,-,80549.530260,1762852,41933215,OTH,normal,0
1,udp,-,0.000000,0,0,S0,normal,0
2,tcp,-,0.000000,0,0,OTH,normal,0
3,tcp,-,0.000000,0,0,OTH,normal,0
4,udp,dns,0.000549,0,298,SHR,normal,0


In [127]:
network21 = network21[network_columns]
network21.rename(columns={"conn_state": "connection"}, inplace=True)
network21.head()

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,udp,dns,0.000008,80,112,SF,xss,1
1,tcp,http,1.208269,380,724,SF,xss,1
2,udp,dns,0.008457,80,112,SF,xss,1
3,tcp,http,1.022216,2074,1211,SF,xss,1
4,tcp,http,1.071203,1816,1125,SF,xss,1


In [128]:
unsw = unsw[unsw_columns]
unsw.rename(columns={
    "dur": "duration",
    "sbytes": "src_bytes",
    "dbytes": "dst_bytes",
    "state": "connection",
    "attack_cat": "type",
}, inplace=True)
unsw.head()

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,tcp,-,0.121478,258,172,FIN,Normal,0
1,tcp,-,0.649902,734,42014,FIN,Normal,0
2,tcp,-,1.623129,364,13186,FIN,Normal,0
3,tcp,ftp,1.681642,628,770,FIN,Normal,0
4,tcp,-,0.449454,534,268,FIN,Normal,0


In [129]:
network1["connection"].value_counts()

connection
S0        635515
REJ       145597
OTH       132341
RSTOS0     48693
SHR        21159
SF         12412
RSTO        3405
S1           518
RSTRH        211
RSTR         181
S3            50
SH            45
S2            25
Name: count, dtype: int64

In [130]:
network21["connection"].value_counts()

connection
SF        964885
S0          8926
S1          8916
S2          7437
S3          5959
SHR         1610
SH          1176
OTH          973
REJ           84
RSTR          23
RSTO           8
RSTRH          2
RSTOS0         1
Name: count, dtype: int64

In [131]:
unsw["connection"].value_counts()

connection
INT    82275
FIN    77825
CON    13152
REQ     1991
RST       83
ECO       12
PAR        1
URN        1
no         1
Name: count, dtype: int64

In [132]:
connection_mapping = {
    "SF": "Finished",          # Connection successfully established and completed
    "S0": "Interrupted",       # SYN sent but no response received
    "S1": "Interrupted",       # Connection established but no data sent
    "S2": "Interrupted",       # Connection initiated but not completed
    "S3": "Interrupted",       # Connection partially established
    "OTH": "Other",            # Other
    "SH": "Connecting",        # SYN received, waiting for ACK
    "SHR": "Connecting",       # Shared handshake state
    "RSTO": "Reset",           # Reset after connection was open
    "RSTR": "Reset",           # Connection reset
    "RSTOS0": "Reset",         # Reset after SYN sent
    "RSTRH": "Reset",          # Reset with retransmission
    "REJ": "Interrupted",      # Connection rejected by the destination
    "FIN": "Finished",         # Connection gracefully terminated
    "INT": "Interrupted",      # Interrupted connection
    "CON": "Connecting",       # Connection currently active
    "ECO": "Request",          # Echo request (e.g., ICMP ping)
    "REQ": "Request",          # Request-based connection (e.g., HTTP, DNS)
    "RST": "Reset",            # Reset connection
    "PAR": "Other",            # Undefined or parameter-related connection
    "URN": "Other",            # Undefined or urgent connection
    "no": "Other"              # Unknown connection
}

network1["connection"] = network1["connection"].map(connection_mapping)
network21["connection"] = network21["connection"].map(connection_mapping)
unsw["connection"] = unsw["connection"].map(connection_mapping)


In [133]:
type_mapping = {
    "xss": "Cross-Site Scripting",
    "normal": "Normal",
    "scanning": "Scanning",
}

network1["type"] = network1["type"].map(type_mapping)
network21["type"] = network21["type"].map(type_mapping)

In [134]:
# Combine the datasets
data = pd.concat([network1, network21, unsw], ignore_index=True)

In [135]:
data["connection"].value_counts()

connection
Finished       1055122
Interrupted     895302
Other           133317
Reset            52607
Connecting       37142
Request           2003
Name: count, dtype: int64

In [136]:
data["type"].value_counts()

type
Cross-Site Scripting    985593
Scanning                791465
Normal                  279094
Generic                  40000
Exploits                 33393
Fuzzers                  18184
DoS                      12264
Reconnaissance           10491
Analysis                  2000
Backdoor                  1746
Shellcode                 1133
Worms                      130
Name: count, dtype: int64

# Canbiar procesar dato (Por Cesar)
Porfavor Revisar 

In [137]:
data["service"].value_counts()

service
-           1065797
http         698149
dns          394597
smtp           5058
ftp-data       3995
ftp            3441
ssl            1487
ssh            1302
pop3           1105
dhcp            178
gssapi           91
smb              81
snmp             80
ntlm             71
irc              25
radius           17
dce_rpc          16
socks             1
imap              1
krb_tcp           1
Name: count, dtype: int64

In [138]:
# Agrupaar por protocolo y contar cuantos son ataques y cuales no (0=no, 1=si)
data.groupby('proto')['label'].value_counts()

proto      label
3pc        1        100
a/n        1        100
aes-sp3-d  1        100
any        1        300
argus      1         98
                   ... 
wsn        1        100
xnet       1         99
xns-idp    1         99
xtp        1        100
zero       1        100
Name: count, Length: 137, dtype: int64

In [139]:
# Exporta a un Csv
data.groupby('proto')['label'].value_counts().to_csv('protocolos_atackes.csv', header=['Numero'])

In [140]:
#Agrupaar por servicios y contar cuantos son ataques y cuales no (0=no, 1=si) 
data.groupby('service')['label'].value_counts()

service   label
-         1        872723
          0        193074
dce_rpc   1            16
dhcp      1            99
          0            79
dns       1        328287
          0         66310
ftp       1          2223
          0          1218
ftp-data  0          2552
          1          1443
gssapi    1            87
          0             4
http      1        686541
          0         11608
imap      1             1
irc       1            25
krb_tcp   1             1
ntlm      1            67
          0             4
pop3      1          1101
          0             4
radius    1            15
          0             2
smb       1            77
          0             4
smtp      1          3479
          0          1579
snmp      1            79
          0             1
socks     1             1
ssh       0          1291
          1            11
ssl       0          1364
          1           123
Name: count, dtype: int64

In [141]:
# Exporta a un Csv
data.groupby('service')['label'].value_counts().to_csv('servicios_atace.csv', header=['Count'])

In [142]:
# Saca Porcentajes que representan ataques segun protocolo
total_counts = data["proto"].value_counts().rename("total")

# volver los tipos de si es o no un ataques en columnas
label_counts = data.groupby('proto')['label'].value_counts().unstack(fill_value=0)

# calcula porcentaje
label_counts["Porcentajes de ataques"] = (label_counts[1] / total_counts * 100).round(2)

# Combina el resultado
result = pd.concat([total_counts, label_counts], axis=1)

# Guarda en csv
result.to_csv("protocolo_ataques_porcentajes.csv", float_format="%.2f")


print(result)

          total       0        1  Porcentajes de ataques
proto                                                   
tcp     1687668  159679  1527989                   90.54
udp      447821  109488   338333                   75.55
unas      12084       0    12084                  100.00
icmp       7907    6985      922                   11.66
arp        2859    2859        0                    0.00
...         ...     ...      ...                     ...
argus        98       0       98                  100.00
rdp          98       0       98                  100.00
netblt       98       0       98                  100.00
igmp         18      18        0                    0.00
rtp           1       1        0                    0.00

[133 rows x 4 columns]


In [143]:
# Calcula el total por protocolos
total_counts = data["proto"].value_counts().rename("total")

# Vuelve cada tipo de ataques en una columna y las cuenta
label_counts = data.groupby('proto')['type'].value_counts().unstack(fill_value=0)

# Transforma el recuento en porcentaje segun el total del protocolo
porcentaje_por_protocolos = label_counts.div(total_counts, axis=0) * 100  # Axis=0 aligns by index (service)

# Redondea a 2 decimales
porcentaje_por_protocolos = porcentaje_por_protocolos.round(2)

# Se agrega la columna
porcentaje_por_protocolos.insert(0, 'total', total_counts)

# exportas
porcentaje_por_protocolos.to_csv("proto_tipo_porcentaje.csv", float_format="%.2f")

# Display
print(porcentaje_por_protocolos)

type       total  Analysis  Backdoor  Cross-Site Scripting    DoS  Exploits  \
proto                                                                         
3pc          100      5.00      5.00                   0.0  34.00     44.00   
a/n          100      5.00      5.00                   0.0  34.00     44.00   
aes-sp3-d    100      5.00      5.00                   0.0  34.00     44.00   
any          300      5.00      5.00                   0.0  34.00     44.00   
argus         98      5.10      4.08                   0.0  34.69     44.90   
...          ...       ...       ...                   ...    ...       ...   
wsn          100      5.00      5.00                   0.0  34.00     44.00   
xnet          99      5.05      4.04                   0.0  34.34     44.44   
xns-idp       99      5.05      4.04                   0.0  34.34     44.44   
xtp          100      5.00      5.00                   0.0  34.00     44.00   
zero         100      5.00      5.00                

In [144]:
# Cuenta el total de servicios
total_counts = data["service"].value_counts().rename("total")

#  transformar si es o no un ataque en  columnas
label_counts = data.groupby('service')['label'].value_counts().unstack(fill_value=0)

# calcula porcentaje
label_counts["Porcentajes de ataques"] = (label_counts[1] / total_counts * 100).round(2)

# los une 
result = pd.concat([total_counts, label_counts], axis=1)

# exporta
result.to_csv("servicios_ataques_porcenaje.csv", float_format="%.2f")


print(result)

            total       0       1  Porcentajes de ataques
service                                                  
-         1065797  193074  872723                   81.88
http       698149   11608  686541                   98.34
dns        394597   66310  328287                   83.20
smtp         5058    1579    3479                   68.78
ftp-data     3995    2552    1443                   36.12
ftp          3441    1218    2223                   64.60
ssl          1487    1364     123                    8.27
ssh          1302    1291      11                    0.84
pop3         1105       4    1101                   99.64
dhcp          178      79      99                   55.62
gssapi         91       4      87                   95.60
smb            81       4      77                   95.06
snmp           80       1      79                   98.75
ntlm           71       4      67                   94.37
irc            25       0      25                  100.00
radius        

In [145]:
# cuenta por tipo de servicio
total_counts = data["service"].value_counts().rename("total")

# transforma los tipos  de ataque en columna
tipo_cuenta = data.groupby('service')['type'].value_counts().unstack(fill_value=0)

# Transforma el tipo en columnas
porcentage_por_tipo = tipo_cuenta.div(total_counts, axis=0) * 100  # Axis=0 aligns by index (service)

# redondea
porcentage_por_tipo = porcentage_por_tipo.round(2)

# se agrega columna
porcentage_por_tipo.insert(0, 'total', total_counts)

# exporta a cV
porcentage_por_tipo.to_csv("servicio_tipo_porcentaje.csv", float_format="%.2f")

# Display
print(porcentage_por_tipo)

type        total  Analysis  Backdoor  Cross-Site Scripting    DoS  Exploits  \
service                                                                        
-         1065797      0.13      0.15                  2.70   0.98      1.61   
dce_rpc        16      0.00      0.00                  0.00   0.00      0.00   
dhcp          178      0.00      0.00                  0.00  18.54     29.21   
dns        394597      0.00      0.00                 72.02   0.03      0.05   
ftp          3441      0.00      0.15                  0.00   1.45     46.93   
ftp-data     3995      0.00      0.00                  0.00   0.20     35.92   
gssapi         91      0.00      0.00                  0.00   0.00      0.00   
http       698149      0.08      0.01                 96.35   0.17      1.24   
imap            1      0.00      0.00                  0.00   0.00      0.00   
irc            25      0.00     44.00                  0.00  16.00     40.00   
krb_tcp         1      0.00      0.00   

In [146]:
porcentage_por_tipo.columns.drop('total')
ranked= porcentage_por_tipo.rank(axis=1, ascending=False, method='min').astype(int)
print(ranked)

type      total  Analysis  Backdoor  Cross-Site Scripting  DoS  Exploits  \
service                                                                    
-             1        10         9                     4    7         5   
dce_rpc       2         3         3                     3    3         3   
dhcp          1         7         7                     7    4         3   
dns           1        10        10                     2    8         7   
ftp           1         9         7                     9    5         2   
ftp-data      1         5         5                     5    4         3   
gssapi        2         4         4                     4    4         4   
http          1         8        12                     2    6         4   
imap          2         3         3                     3    3         3   
irc           3         5         1                     5    4         2   
krb_tcp       2         3         3                     3    3         3   
ntlm        

In [147]:
data

,proto,service,duration,src_bytes,dst_bytes,connection,type,label
0,tcp,-,80549.530260,1762852,41933215,Other,Normal,0
1,udp,-,0.000000,0,0,Interrupted,Normal,0
2,tcp,-,0.000000,0,0,Other,Normal,0
3,tcp,-,0.000000,0,0,Other,Normal,0
4,udp,dns,0.000549,0,298,Connecting,Normal,0
...,...,...,...,...,...,...,...,...
2175488,udp,dns,0.000009,114,0,Interrupted,Generic,1
2175489,tcp,-,0.505762,620,354,Finished,Shellcode,1
2175490,udp,dns,0.000009,114,0,Interrupted,Generic,1
2175491,udp,dns,0.000009,114,0,Interrupted,Generic,1
